In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# 0. 데이터 정렬 (중요: 회원별, 월별 순서가 맞아야 rolling이 정확합니다)
df = df.sort_values(by=['발급회원번호', '기준년월'])

# -------------------------------------------------------
# 1. 이동 기울기(Rolling Slope) 계산 함수
# -------------------------------------------------------
def calc_slope(y):
    # 데이터가 2개 미만이면 기울기를 구할 수 없음
    if len(y) < 2 or np.sum(y) == 0: 
        return 0.0
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return slope

# 최근 3개월(window=3)의 데이터를 사용하여 기울기 산출
# min_periods=2: 데이터가 2개만 쌓여도 기울기 계산 시작
group = df.groupby('발급회원번호')

df['Slope_Spend'] = group['이용금액_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Balance'] = group['잔액_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Count'] = group['이용건수_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)

# 결측치(첫 달 등)는 0으로 채움
df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']] = df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']].fillna(0)

# -------------------------------------------------------
# 2. 기울기를 '이탈 점수'로 변환 (모든 행에 대해 계산)
# -------------------------------------------------------
scaler = MinMaxScaler()

def convert_slopes_to_scores(dataframe):
    # 음수(감소)인 경우만 추출하여 양수로 변환 (가장 많이 줄어든 게 가장 큰 값)
    # 7월부터 12월까지 모든 행의 기울기를 한꺼번에 스케일링하여 상대적 위치 파악
    temp_scores = {}
    for col in ['Slope_Spend', 'Slope_Balance', 'Slope_Count']:
        neg_val = dataframe[col].apply(lambda x: -x if x < 0 else 0)
        temp_scores[col] = scaler.fit_transform(neg_val.values.reshape(-1, 1)).flatten()
    return temp_scores

# 정규화된 점수 계산 (0.0 ~ 1.0)
norm_scores = convert_slopes_to_scores(df)

# -------------------------------------------------------
# 3. 가중치 적용 및 최종 점수(Churn_Score) 산출
# -------------------------------------------------------
W_SPEND, W_BALANCE, W_COUNT, W_RISK = 40, 30, 20, 10

# 리스크 점수 (연체나 거절이 있으면 1점, 없으면 0점 -> 나중에 가중치 10 곱함)
df['Risk_Flag'] = np.where(
    (df.get('연체잔액_B0M', 0) > 0) | (df.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

# 최종 스코어 합산
df['Churn_Score'] = (
    (norm_scores['Slope_Spend'] * W_SPEND) + 
    (norm_scores['Slope_Balance'] * W_BALANCE) + 
    (norm_scores['Slope_Count'] * W_COUNT) + 
    (df['Risk_Flag'] * W_RISK)
)

df['Churn_Score'] = df['Churn_Score'].round(1)

# -------------------------------------------------------
# 4. 결과 확인 (최신 월인 12월 데이터만 보기)
# -------------------------------------------------------
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

print(f"\n[{target_month} 기준 이탈 위험 점수 상위 10명]")
cols_view = ['발급회원번호', '기준년월', 'Slope_Spend', 'Churn_Score']
print(df_final[cols_view].sort_values('Churn_Score', ascending=False).head(10))

In [ ]:
# 1. 확인하고 싶은 발급회원번호 설정 (실제 데이터에 있는 번호를 넣으세요)
target_id = df['발급회원번호'].iloc[89765]  # 예시로 첫 번째 회원의 ID를 가져옵니다.

# 2. 해당 회원의 데이터만 추출하여 주요 컬럼 확인
# 새로 만든 컬럼들 + 기준이 되는 컬럼들을 함께 봅니다.
view_cols = [
    '기준년월', 
    '이용금액_신용_B0M', 'Slope_Spend',   # 실제 소비량과 그 기울기
    '잔액_B0M', 'Slope_Balance',        # 잔액과 그 기울기
    'Risk_Flag',                       # 리스크 여부
    'Churn_Score'                      # 최종 이탈 위험 점수
]

customer_history = df[df['발급회원번호'] == target_id][view_cols].sort_values('기준년월')

print(f"🔎 [회원번호: {target_id}] 의 6개월간 이탈 징후 변화")
print("-" * 100)
print(customer_history)
print("-" * 100)

# 3. (팁) 점수가 가장 많이 변한 달 찾기
diff = customer_history['Churn_Score'].max() - customer_history['Churn_Score'].min()
print(f"💡 해당 고객의 6개월간 점수 변동폭: {diff:.1f}점")